In [1]:
import plotly_express as px
import pandas as pd
import matplotlib.pyplot as plt

import yfinance as yf
from backtesting import Backtest, Strategy
import pandas_ta as ta

from backtesting.lib import crossover
import math

In [2]:
# If you want to check if the strategy is executing trades correctly, use this to validate the data that can be run with the backtest and can handle the plot.
ticker = "usdjpy=X"
data = yf.download(ticker, period="1y", interval="1h")


[*********************100%%**********************]  1 of 1 completed


In [ ]:
# , delimiter='\t', names=['Open', 'High', 'Low', 'Close', 'Volume'], header=0

In [5]:
# Importing through a CSV that has more data which was downloaded externally.
df = pd.read_csv('../Data/Darwinex/GBPJPY60.csv', header=None, names=['Date', 'Open', 'High', 'Low','Close','Volume'])
# df.index = pd.to_datetime(df.index)


In [10]:
df

,2013.08.21,09:00,152.669,152.850,152.573,152.823,4543
0,2013.08.21,10:00,152.824,152.947,152.419,152.864,5086
1,2013.08.21,11:00,152.872,152.991,152.706,152.829,3732
2,2013.08.21,12:00,152.828,152.961,152.683,152.713,3534
3,2013.08.21,13:00,152.712,153.005,152.646,152.956,3639
4,2013.08.21,14:00,152.958,153.058,152.925,152.952,3292
...,...,...,...,...,...,...,...
64996,2024.02.15,22:00,188.881,188.962,188.874,188.889,6730
64997,2024.02.15,23:00,188.891,188.949,188.867,188.905,3795
64998,2024.02.16,00:00,188.727,188.900,188.719,188.840,1501
64999,2024.02.16,01:00,188.841,188.953,188.812,188.928,3260


In [ ]:
df['ATR'] = ta.atr(pd.Series(df.High), 
                          pd.Series(df.Low), 
                          pd.Series(df.Close), 
                          length=14)

df['ATR']  = round(df['ATR'], 4)

In [ ]:
# csv slice by date
start_date = '2023-01-01'
end_date = '2023-01-01'
df = df.loc[start_date:end_date]

# Write out strategy here to figure out the logic

In [ ]:
class Strat(Strategy):
    def init(self):
        pass

    def next(self):
        pass

bt = Backtest(df_slice, Strat, cash=10_000)
bt.run()
bt.plot()

In [ ]:
# Position sizing for JPY pairs
# position sizing logic
        account_balance = math.floor(self.equity)
        risk_per_trade = account_balance * 0.01
        stop_loss = self.data.Low[-1] - self.atr[-1]
        stop_in_pips = abs(current_close - stop_loss) * 100  # Calculate stop loss in pips        
        # Adjust position size based on the risk per trade and stop loss distance
        position_size = math.floor((risk_per_trade / (stop_in_pips * 10)) * 1000) 
        take_profit = ((stop_in_pips / 100) * 1.2) + current_close
        result = position_size 

In [ ]:
# position sizing logic for pairs != JPY
        account_balance = math.floor(self.equity)
        dollar_risk_per_trade = account_balance * 0.01
        stop_loss = self.data.Low[-1] - self.atr[-1]
        stop_in_pips = round((current_close - stop_loss) * 10000, 2)  # Pips calculation is correct
        print(stop_in_pips * position_size)
        # Corrected position size calculation
        position_size = math.floor((dollar_risk_per_trade / (stop_in_pips * 10)) * 1000)

In [5]:
class Strat(Strategy):
    def init(self):
        # Calculate the 50-period EMA and 14-period ATR and check engulfing        
        self.ema = self.I(ta.ema, pd.Series(self.data.Close), length=50)
        self.atr = self.I(ta.atr, 
                          pd.Series(self.data.High), 
                          pd.Series(self.data.Low), 
                          pd.Series(self.data.Close), 
                          length=14)
               
        self.pullback = False
        self.pullback_count = 0
        self.consolidation_high = 0    

        self.position_status = False

        self.custom_trades_log = []
        
    def next(self):
        
        current_ema = self.ema[-1]
        current_close = self.data.Close[-1]
        current_open = self.data.Open[-1]
        current_high = self.data.High[-1]        
        
        # Buy logic
        if crossover(current_close, current_ema) or (current_close < current_ema):
            self.pullback = False
            self.pullback_count = 0
            self.consolidation_high = 0

        # Pullback logic
        if current_high > self.consolidation_high:
            self.consolidation_high = max(self.consolidation_high, current_high)
        if current_high < self.consolidation_high:
            if current_close < self.consolidation_high and current_close < current_open:
                self.pullback_count += 1

            if self.pullback_count >= 2:
                self.pullback = True
            if not self.position:
                if current_close < self.consolidation_high and self.pullback:
                    if current_close > self.data.High[-2]:
                        self.pullback = False
                        self.pullback_count = 0
                        self.consolidation_high = 0
                        sl = self.data.Low[-1] - self.atr[-1]
                        sl_pips = self.data.Close - sl
                        tp = self.data.Close[-1] + (sl_pips * 1.2)
                        
                        # Log the initiation of a trade
                        self.custom_trades_log.append({
                            'entry_time': self.data.index[-1],
                            'entry_price': current_close,
                            'direction': 'BUY',
                            'sl': sl,
                            'tp': tp,
                            'exit_time': None,  # To be merged from backtesting _trades
                            'exit_price': None,  # To be merged from backtesting _trades
                        })
                        
                        # Execute the trade
                        self.buy(sl=sl, tp=tp)        
                    

In [40]:
bt = Backtest(df, Strat, cash=100000, margin=0.02)
stats = bt.run()
print(stats)


Start                     2008-01-25 02:00:00
End                       2024-02-02 19:00:00
Duration                   5852 days 17:00:00
Exposure Time [%]                       0.949
Equity Final [$]                     2.249233
Equity Peak [$]                     114445.12
Return [%]                         -99.997751
Buy & Hold Return [%]              -11.587313
Return (Ann.) [%]                  -41.620983
Volatility (Ann.) [%]               42.259608
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -99.998035
Avg. Drawdown [%]                  -39.568473
Max. Drawdown Duration     5847 days 05:00:00
Avg. Drawdown Duration     1949 days 05:00:00
# Trades                                   73
Win Rate [%]                        34.246575
Best Trade [%]                       3.159005
Worst Trade [%]                     -1.761257
Avg. Trade [%]                    

In [ ]:
stats._trades

In [8]:
bt.plot()

d:\Apps\Backtesting\sandbox\venv\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
d:\Apps\Backtesting\sandbox\venv\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
d:\Apps\Backtesting\sandbox\venv\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
d:\Apps\Backtesting\sandbox\venv\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1360', ...)


### Merge the _trades to my custom trade log to perform additional analysis

In [9]:
# Creates the custom trade log into a df and merges the exit information from _trades to this custom df.
custom_df = pd.DataFrame(stats._strategy.custom_trades_log)
custom_df['exit_time'] = stats._trades['ExitTime']
custom_df['exit_price'] = stats._trades['ExitPrice']


In [10]:
# Pips calculations
custom_df['stop_pips'] = round((custom_df['entry_price'] - custom_df['sl']) * 100, 2)
custom_df['result_pips'] = round((custom_df['exit_price'] - custom_df['entry_price']) * 100, 2)
custom_df['rr'] = round(custom_df['result_pips'] / custom_df['stop_pips'], 2)


In [15]:
initial_equity = 100000.0
# Initialize account equity and risk for the first row only
custom_df.at[0, 'account_equity'] = initial_equity
custom_df.at[0, 'risk'] = initial_equity * 0.01
custom_df['pnl'] = 0.0  # Initialize pnl column

for index, row in custom_df.iterrows():
    # Calculate pnl from the first row
    if index > 0:
        # Use previous row's account equity to calculate risk for the current trade
        custom_df.at[index, 'risk'] = custom_df.at[index - 1, 'account_equity'] * 0.01
    
    # pnl calculation includes the first trade
    pnl = custom_df.at[index, 'risk'] * row['rr']
    custom_df.at[index, 'pnl'] = pnl
    
    
    if index == 0:
        custom_df.at[index, 'account_equity'] += pnl  
    else:
        custom_df.at[index, 'account_equity'] = custom_df.at[index - 1, 'account_equity'] + pnl

# Ensure data types
custom_df['account_equity'] = custom_df['account_equity'].astype(float)
custom_df['risk'] = custom_df['risk'].astype(float)
custom_df['pnl'] = custom_df['pnl'].astype(float)


In [16]:
custom_df.head()

,entry_time,entry_price,direction,sl,tp,exit_time,exit_price,stop_pips,result_pips,rr,account_equity,risk,pnl
0,2023-02-21 21:00:00+00:00,134.992996,BUY,134.704838,"[134.13638772752935, 134.03798684862312, 134.2...",2023-02-22 00:00:00+00:00,134.704838,28.82,-28.82,-1.0,99000.000000,1000.000000,-1000.000000
1,2023-02-22 08:00:00+00:00,134.914993,BUY,134.377014,"[134.45177353928383, 134.3533726603776, 134.58...",2023-02-22 12:00:00+00:00,134.377014,53.80,-53.80,-1.0,98010.000000,990.000000,-990.000000
2,2023-02-22 23:00:00+00:00,134.936996,BUY,134.607890,"[134.19672565312084, 134.09832477421457, 134.3...",2023-02-23 13:00:00+00:00,135.331925,32.91,39.49,1.2,99186.120000,980.100000,1176.120000
3,2023-02-24 21:00:00+00:00,136.485001,BUY,136.141605,"[133.90427170320032, 133.8058708242941, 134.03...",2023-02-27 00:00:00+00:00,136.141605,34.34,-34.34,-1.0,98194.258800,991.861200,-991.861200
4,2023-02-27 09:00:00+00:00,136.384995,BUY,135.977285,"[134.0014490092265, 133.90304813032026, 134.13...",2023-02-27 13:00:00+00:00,135.977285,40.77,-40.77,-1.0,97212.316212,981.942588,-981.942588


In [29]:
total_trades = len(custom_df)
winning_trades = (custom_df['rr'] > 0).sum()
win_rate = f'{round((winning_trades / total_trades) * 100, 2)}%'
average_win_r = round(custom_df[custom_df['rr'] > 0]['rr'].mean(), 2)


In [38]:

average_rr


1.2

In [ ]:

# plot own equity curve based on backtesting data for data more than 10K
equity_curve = stats._equity_curve['Equity']

plt.figure(figsize=(10, 6))
plt.plot(equity_curve, label='Equity Curve', lw=1)  # lw is line width
plt.title('Equity Curve')
plt.xlabel('Time')
plt.ylabel('Equity')
plt.legend()
plt.grid(False)
plt.show()


In [33]:

equity_curve = stats._equity_curve['Equity'].reset_index()

fig = px.line(custom_df, x='entry_time', y='account_equity', labels={'index': 'Time'}, title='Strategy Performance')
fig.update_layout(height=600, xaxis_title='Time', yaxis_title='Equity', legend_title='Legend')
fig.show()
